# 1- PLATEFORME D'ANALYSE DES ACTIONS

Ce document sert à effectuer une analyse des mesures statistiques des titres côtés sur la BRVM. La première étape est d'initialiser les données sur la plateforme afin de pouvoir effectuer les analyses.

* Pour exécuter les lignes de code, il suffit de taper sur Shift puis Entrée, vous pouvez aussi utiliser la fonction Run All Cells.

## 1.1 - INITIALISATION DES DONNÉES

Cette étape consiste à importer les modules qui vont servir à une exécution correcte du script.

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import edhec_risk_kit as erk
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# Modules for the buttons and widgets 
import ipywidgets as widgets
from ipywidgets import Button, Layout, Text,  HBox
from ipywidgets import Dropdown
from ipywidgets import Output
from IPython.display import display

%load_ext autoreload
%autoreload 2
%matplotlib inline

/workspaces/Financial-Risk-Assessment-Platform/edhec_risk_kit.py:36: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  if filetype is "returns":
/workspaces/Financial-Risk-Assessment-Platform/edhec_risk_kit.py:39: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  elif filetype is "nfirms":
/workspaces/Financial-Risk-Assessment-Platform/edhec_risk_kit.py:42: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  elif filetype is "size":


ModuleNotFoundError: No module named 'ipywidgets'

Code à exécuter une fois puis supprimer le code

In [ ]:
 conda install -c conda-forge ta-lib

Retrieving noticesdone
Channels:
 - conda-forge
 - defaults
Platform: osx-64
doneecting package metadata (repodata.json): - 
doneing environment: | 

## Package Plan ##

  environment location: /Users/arambengue/anaconda3

  added / updated specs:
    - ta-lib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.1.31  |       h8857fd0_0         155 KB  conda-forge
    certifi-2025.1.31          |     pyhd8ed1ab_0         159 KB  conda-forge
    conda-25.1.1               |  py311h6eed73b_1         1.2 MB  conda-forge
    openssl-3.4.1              |       hc426f3f_0         2.5 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.9 MB

The following packages will be UPDATED:

  ca-certificates    pkgs/main::ca-certificates-2024.12.31~ --> conda-forge::ca-certificates-2025.1.31-h8857f

### 1.1-A) Importer des données historiques à partir d'Excel

**Importer les données d'Excel**

In [ ]:
import pandas as pd
from PyQt5.QtWidgets import QApplication, QFileDialog

def upload_excel():
    """Prompts the user to select an Excel file and returns the file path."""
    app = QApplication([])
    file_path, _ = QFileDialog.getOpenFileName(None, "Select Excel File", "", "Excel files (*.xlsx *.xls)")
    app.exit()
    return file_path

def read_excel(file_path):
  """Reads the selected Excel file into a pandas DataFrame."""
  if file_path:
    data = pd.read_excel(file_path,header=0,index_col=0)
    return data
  else:
    print("No file selected")
    
# Usage
file_path = upload_excel()
if file_path:
    data = read_excel(file_path)
    data.index = pd.to_datetime(data.index, dayfirst=True)
    print(data.head())


2025-02-20 00:45:31.396 python[23845:659008] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-20 00:45:32.425 python[23845:659008] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'


            CABC     FTSC  NEIC  NTLC     SEMC  SIVC    SLBC    SMBC    STBC  \
Date                                                                           
2017-12-29  1105  4925.00    37  1900  346.875   400  125000  1905.0  3600.0   
2018-01-02  1150  4875.00    37  1990  346.875   420  125000  2000.0  3600.0   
2018-01-03  1200  4823.75    37  1990  346.875   420  125000  2125.0  3350.0   
2018-01-04  1290  4823.75    37  2100  362.500   415  124000  2225.0  3350.0   
2018-01-05  1205  4625.00    37  2000  362.500   410  124000  2212.5  3350.0   

            TTRC  ...  BRVM-10  BRVM-30  BRVM - PRESTIGE  BRVM - AGRICULTURE  \
Date              ...                                                          
2017-12-29   490  ...   219.65      NaN              NaN              183.48   
2018-01-02   490  ...   215.12      NaN              NaN              180.81   
2018-01-03   490  ...   213.80      NaN              NaN              184.01   
2018-01-04   490  ...   212.99      NaN

## 1.2 - MESURES STATISTIQUES DES ACTIONS

### 1.2-A) CALCUL DES MESURES STATISTIQUES DES TITRES INDIVIDUELS

- Pour le bon fonctionnement du code, il faudra installer conda grâce au code " install -c conda-forge ta-lib" , après l'avoir installer, il faudra supprimer la ligne de code.
- Les calculs ne pourront se faire sur une base de données de moins d'un an. Ainsi, assurez-vous d'utiliser une base sur 2 à N années.
- Vous avez la possibilité de choisir votre taux sans risque et la périodicité en fonction de votre base. Si la base est journalière, choisissez une périodicité journalière.
- Le code du jensen_alpha utilise directement les données de l'indice BRVM-C. Ainsi, assurez-vous de nommer l'indice par BRVM - COMPOSITE sur votre base de données.

- Les rendements ordinaires (simple returns (𝑃𝑡−𝑃𝑡−1)/𝑃𝑡−1) ne sont pas additifs dans le temps, c'est-à-dire que le rendement ordinaire sur plusieurs périodes (multiperiod return) n'est pas exactement égal à la somme des rendements ordinaires unipériodes mais il n'est qu'approximativement égal lorsque les rendements sont "faibles" (proches de 0, généralement lorsque l'unité de temps d'observation est courte, en jour, heures ou minutes). Par conséquent, dans le cadre de cette analyse, vous avez le choix d'utiliser le rendement composé en continu ou rendement logarithmique (ln(𝑃𝑡/𝑃𝑡−1)) = 𝑝𝑡−𝑝𝑡−1 𝑜ù 𝑝𝑡 = ln(𝑃𝑡)). En effet, les rendements logarithmiques sont additifs en série temporelle et leur utilisation dans la modélisation financière est utile du fait que notre contexte d’investissement implique généralement le réinvestissement du capital.

- L'Expected Shortfall (ES) est une mesure du risque financier qui vous indique la perte moyenne à laquelle vous pouvez vous attendre dans les pires scénarios. Il se concentre sur la « queue » des rendements des investissements, c'est-à-dire les résultats négatifs extrêmes. Le niveau de significativité pour l'ES est le même que le niveau de confiance pour la VaR, mais l'ES fournit une vision plus complète des pertes potentielles au-delà de ce seuil.
- Le Maximum Drawdown est la perte maximale entre le niveau le plus haut et le niveau le plus bas du cours d'une action. C'est le pire rendement que vous auriez pu obtenir en achetant au plus haut et en vendant au plus bas. Le Drawdown n'est pas tant robuste car il dépend fortement de la granularité (si nous utilisons des données quotidiennes l'impact sera d'autant plus important).Afin de calculer le maximum drawdown, nous créons un indice de richesse (wealth index) en utilisant la fonction cumprod(). On raisonne comme si on investissait 1000$ sur l'actif au pire moment possible (prix le plus élevé) et nous regardons les pics précédents pour déterminer la différence entre les deux (wealth index-previous peak/previous peak).

- Le calcul de la VaR prend en compte l'expansion de Corner Fischer. Il est possible de calculer uniquement la VaR Gaussienne en changeant la fonction avec modified = False. Dans notre cas, nous avons utilisé la VaR de Corner-Fisher car les rendements ne sont généralement pas normaux (Non Gaussian). Utiliser la Gaussian VaR ou la VaR historique ne serait pas appoprié.


In [ ]:
# Definition fonction Log_return et return

def calculate_returns(data, stock, return_type):
  if return_type == 'Rendements Logarithmiques':
    returns = np.log(data[stock].dropna() / data[stock].dropna().shift(1))
  elif return_type == 'Rendements Simples':
    returns = data[stock].dropna().pct_change()
  else:
    raise ValueError("Type de rendement non applicable")
  return returns

return_type_options = ['Rendements Logarithmiques', 'Rendements Simples']
return_type_dropdown = widgets.Dropdown(options=return_type_options, description='Type de rendement:')

stock_options = data.columns.tolist()
stock_dropdown = widgets.Dropdown(options=stock_options, description='Actions:')

# Dropdown pour la sélection de la périodicité des données
per_selection = {
    "Journalier":'252',
    "Hebdomadaire":'52', 
    "Mensuelle": '12', 
}


per_options = list(per_selection.keys()) 
per_dropdown = Dropdown(
    description="Périodicité:",
    options=per_options,
    value=per_options[0], 
)

# Dropdown pour la sélection du taux sans risque 
rfr_selection = {
    "5.55":'5.55',
    "5.65":'5.65',
    "5.70":'5.70',
    "5.80":'5.80',
    "5.85":'5.85',
    "5.90":'5.90',
    "5.95":'5.95', 
    "6.00":'6.00', 
    "6.05":'6.05',
    "6.10":'6.10',
    "6.15":'6.15',
    "6.20":'6.20',
    "6.25":'6.25',
    "6.30":'6.30',
    "6.35":'6.35',
    "6.40":'6.40',
    "6.45":'6.45',
    "6.50":'6.50',
    "6.55":'6.55',
    "6.60":'6.60',
    "6.65":'6.65',
    "6.70":'6.70',
    "6.75":'6.75',
    "6.80":'6.80',
    "6.85":'6.85',
    "6.90":'6.90',
}

rfr_options = list(rfr_selection.keys()) 
rfr_dropdown = Dropdown(
    description="Taux sans risque:",
    options=rfr_options,
    value=rfr_options[10], 
)


# Dropdown pour la selection de la date à considérer

# Dropdown pour la sélection du taux sans risque 
date_selection = {
    "2035":'2035',
    "2034":'2034',
    "2033":'2033',
    "2032":'2032',
    "2031":'2031',
    "2030":'2030',
    "2029":'2029', 
    "2028":'2028', 
    "2027":'2027',
    "2026":'2026',
    "2025":'2025',
    "2024":'2024',
    "2023":'2023',
    "2022":'2022',
    "2021":'2021',
    "2020":'2020',
    "2019":'2019',
    "2018":'2018', 
    "2017":'2017', 
    "2016":'2016',
    "2015":'2015',
    "2014":'2014',
    "2013":'2013',
    "2012":'2012',
    "2011":'2011',
    "2010":'2010',
    "2009":'2009',
    "2008":'2008',
    "2007":'2007',
    "2006":'2006',
    "2005":'2005',
    "2004":'2004',
}

date_options = list(date_selection.keys()) 
date_dropdown = Dropdown(
    description="Filtre Date:",
    options=date_options,
    value=date_options[12], 
)



# Calcul des mesures statistiques

def rend():
    ret= calculate_returns(data, stock_dropdown.value, return_type_dropdown.value).loc[date_dropdown.value:,]
    mean_ret= (ret.mean()*100).round(5)
    ann_ret=(erk.annualize_rets(ret,int(per_selection[per_dropdown.value]))*100).round(5)
    comp_ret=(((ret+1).prod()-1)*100).round(5)
    print(f"Action: {stock_dropdown.value}")
    print(f"Rendement moyen {per_dropdown.value} (en %):", mean_ret)
    print("Rendement moyen annuel (en %):", ann_ret)
    print("Rendement géométrique (en %):", comp_ret)
    
rend_button = widgets.Button(description="Rendements",button_style='info')
rend_button.layout.width = '210px'
rend_button.layout.height = '40px'

output9 = widgets.Output()


    
def on_button_clicked9(b):
 with output9:
    output9.clear_output()
    rend()
      
rend_button.on_click(on_button_clicked9)


def volatility():
    ret= calculate_returns(data, stock_dropdown.value, return_type_dropdown.value).loc[date_dropdown.value:,]
    vol = round(ret.std()*100,5)
    ann_vol=round(erk.annualize_vol(ret,int(per_selection[per_dropdown.value]))*100,5)
    print(f"Action: {stock_dropdown.value}")
    print(f"Volatilité {per_dropdown.value} (en %):", vol)
    print("Volatilité annuelle (en %):", ann_vol)

vol_button = Button(description="Volatilités", button_style='info')
vol_button.layout.width = '210px'
vol_button.layout.height = '40px'
output10 = widgets.Output()

def on_button_clicked10(b):
 with output10:
    output10.clear_output()
    volatility()
      
vol_button.on_click(on_button_clicked10)

def downside():
    ret= calculate_returns(data, stock_dropdown.value, return_type_dropdown.value).loc[date_dropdown.value:,]
    semidev = round(erk.semideviation(ret)*100,5)
    Var95= round(erk.var_gaussian(ret,level=5, modified=True)*100,5)
    Var99=round(erk.var_gaussian(ret,level=1, modified=True)*100,5)
    drawdown=erk.drawdown(ret)
    mean_drawdown=drawdown.loc[:,'Drawdown']
    draw= round(mean_drawdown.min()*100*-1,5)
    is_beyond95 = ret <= -erk.var_gaussian(ret, level=5, modified=True)
    shortfall95=round(-((0.05*ret[is_beyond95].mean())-0.95*erk.var_gaussian(ret, level=5, modified=True))*100,5)
    is_beyond99 = ret <= -erk.var_gaussian(ret, level=1, modified=True)
    shortfall99=round(-((0.01*ret[is_beyond99].mean())-0.99*erk.var_gaussian(ret, level=1, modified=True))*100,5)
    print(f"Action: {stock_dropdown.value}")
    print(f"Semideviation {per_dropdown.value} (en %):", semidev)
    print(f"Value at Risk {per_dropdown.value} seuil de 5% (en %):", Var95)
    print(f"Expected Shortfall {per_dropdown.value} seuil de 5% (en %):",shortfall95)
    print(f"Value at Risk {per_dropdown.value} seuil de 1% (en %):", Var99)
    print(f"Expected Shortfall {per_dropdown.value} seuil de 1% (en %):",shortfall99)
    print(f"Maximum Drawdown {per_dropdown.value} (en %):", draw)
    
    # plot drawdown
    fig1, axes1 = plt.subplots(1, 2, figsize=(10, 4))
    plot_wealth=erk.drawdown(ret)[["Wealth","Previous Peak"]].plot(ax=axes1[0])
    axes1[0].set_title('Ancien Pic et Wealth Index')
    axes1[0].set_xlabel('Dates')
    axes1[0].set_ylabel('Cours')
    
    plot_draw=erk.drawdown(ret)[["Drawdown"]].plot(ax=axes1[1])
    axes1[1].set_title('Drawdown')
    axes1[1].set_xlabel('Dates')
    axes1[1].set_ylabel('Cours')
    plt.show()

downside_button = Button(description="Mesure de risque de baisse",button_style='primary')
downside_button.layout.width = '210px'
downside_button.layout.height = '40px'

output11 = widgets.Output()

def on_button_clicked11(b):
 with output11:
    output11.clear_output()
    downside()
      
downside_button.on_click(on_button_clicked11)

def performance():
    ret= calculate_returns(data, stock_dropdown.value, return_type_dropdown.value).loc[date_dropdown.value:,]
    mkt_ret=calculate_returns(data,'BRVM - COMPOSITE', return_type_dropdown.value).loc[date_dropdown.value:,]
    sharpe = round(erk.sharpe_ratio(ret,float(rfr_selection[rfr_dropdown.value])/100,int(per_selection[per_dropdown.value])),5)
    
    # Jensen alpha
    rf_per_period = (1+float(rfr_selection[rfr_dropdown.value])/100)**(1/int(per_selection[per_dropdown.value]))-1
    excess_stock_return = pd.Series(ret - rf_per_period)
    excess_market_return = pd.Series(mkt_ret - rf_per_period)
    beta= stats.linregress(excess_market_return.dropna(), excess_stock_return.dropna())[0]
    jensen_alpha = round(excess_stock_return.mean() - beta * excess_market_return.mean(),5)
    perf=round(jensen_alpha*ret.mean()*100,5)
    # Treynor ratio
    if excess_market_return.var() == 0:
        return None
    model = sm.OLS(excess_stock_return.dropna(), sm.add_constant(excess_market_return.dropna()))
    beta_est = model.fit().params[1]  # Extract beta coefficient
    treynor_ratio = (excess_stock_return.mean()) / beta_est
    # Sortino ratio
    downside_risk = np.sqrt(np.mean(excess_stock_return[excess_market_return < 0] ** 2))
    if downside_risk == 0:
        return None 
    sortino_ratio = (excess_stock_return.mean()) / downside_risk
    print(f"Action: {stock_dropdown.value}")
    print("Ratio de Sharpe:", sharpe)
    print(f"Jensen Alpha {per_dropdown.value}:", f"{jensen_alpha:.5f}")
    print("Performance par rapport au CAPM:", f"{perf:.5f}")
    print("Ratio de treynor:", f"{treynor_ratio:.5f}")
    print("Ratio de Sortino:", f"{sortino_ratio:.5f}")
    
performance_button = Button(description="Mesure de performance",button_style='primary')
performance_button.layout.width = '210px'
performance_button.layout.height = '40px'

output12 = widgets.Output()

def on_button_clicked12(b):
 with output12:
    output12.clear_output()
    performance()

performance_button.on_click(on_button_clicked12)    
    
def stat():
    ret= calculate_returns(data, stock_dropdown.value, return_type_dropdown.value).loc[date_dropdown.value:,]
    mini = (ret*100).min()
    maxi = (ret*100).max()
    quantiles=round(ret.quantile([.05, .25, .5, .75, .95])*100,5)
    print(f"Action: {stock_dropdown.value}")
    print(f"Rendement {per_dropdown.value} Historique Minimal (en %):", f"{mini:.5f}")
    print(f"Rendement {per_dropdown.value} Historique Maximal (en %):", f"{maxi:.5f}")
    print(f"Quantiles (en %): \n{quantiles}")

stat_button = Button(description="Mesures statistiques")
stat_button.style.button_color = 'lightgreen'
stat_button.layout.width = '210px'
stat_button.layout.height = '40px'

output13 = widgets.Output()

def on_button_clicked13(b):
 with output13:
    output13.clear_output()
    stat()

stat_button.on_click(on_button_clicked13)    
    
def distribution():
    ret= calculate_returns(data, stock_dropdown.value, return_type_dropdown.value).loc[date_dropdown.value:,]
    skewness = erk.skewness(ret)
    kurtosis= erk.kurtosis(ret)

    # Create the figure for both plots (optional for better layout)
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # Adjust figure size as needed

    # Create the skewness plot
    plot_skew = sns.histplot(data=ret, kde=True, stat='density', ax=axes[0])
    axes[0].set_title('Skewness Distribution')
    axes[0].set_xlabel('Rendement')
    axes[0].set_ylabel('Densité')

    # Create the kurtosis plot
    plot_kurt = sm.qqplot(ret, line='q', ax=axes[1])
    axes[1].set_title('QQ Plot pour le Kurtosis')
    axes[1].set_xlabel('Quantiles Théoriques')
    axes[1].set_ylabel('Sample Quantiles')

    print(f"Action: {stock_dropdown.value}")
    print("Skewness:", f"{skewness:.5f}")
    print("Kurtosis:", f"{kurtosis:.5f}")
    
    # Display the plots within the notebook (no need for plot_skew.show() or plot_kurt.show())
    plt.tight_layout()  # Adjust spacing between plots (optional)
    plt.show()
    
dist_button = Button(description="Distribution")
dist_button.style.button_color = 'lightgreen'
dist_button.layout.width = '210px'
dist_button.layout.height = '40px'

output14 = widgets.Output()

def on_button_clicked14(b):
 with output14:
    output14.clear_output()
    distribution()
    
dist_button.on_click(on_button_clicked14)    



#### Code For MACD, RSI, BOLLINGER
# Talib installation
# conda install -c conda-forge ta-lib

import talib

def calculate_indicators(ind_data):
  """Calculates RSI, Bollinger Bands, and MACD.

  Args:
    data: A pandas DataFrame containing stock data.

  Returns:
    A pandas DataFrame with additional columns for RSI, Bollinger Bands, and MACD.
  """
  ind_data=data.copy().loc[date_dropdown.value]
  ind_data['RSI'] = talib.RSI(ind_data[stock_dropdown.value], timeperiod=14)
  ind_data['UpperBand'], ind_data['MiddleBand'], ind_data['LowerBand'] = talib.BBANDS(ind_data[stock_dropdown.value], timeperiod=20, nbdevup=2, nbdevdn=2)
  ind_data['MACD'], ind_data['MACDsignal'], ind_data['MACDhist'] = talib.MACD(ind_data[stock_dropdown.value], fastperiod=12, slowperiod=26, signalperiod=9)
  return ind_data

def plot_indicators(ind_data):
  """Plots RSI, Bollinger Bands, and MACD.

  Args:
    df: A pandas DataFrame containing stock data and indicators.
  """
  fig, axs = plt.subplots(3, 1, figsize=(10, 6))

  # RSI
  axs[0].plot(ind_data.index, ind_data['RSI'])  
  axs[0].axhline(70, color='r', linestyle='--')
  axs[0].axhline(30, color='r', linestyle='--')
  axs[0].set_title('RSI')

  # Bollinger Bands
  axs[1].plot(ind_data.index, ind_data['UpperBand'], color='gray', linestyle='--')
  axs[1].plot(ind_data.index, ind_data['MiddleBand'], color='black') 
  axs[1].plot(ind_data.index, ind_data['LowerBand'], color='gray', linestyle='--')
  axs[1].plot(ind_data.index, ind_data[stock_dropdown.value], color='blue') 
  axs[1].set_title('Bollinger Bands')

  # MACD
  axs[2].plot(ind_data.index, ind_data['MACD'], color='blue')
  axs[2].plot(ind_data.index, ind_data['MACDsignal'], color='red')
  axs[2].bar(ind_data.index, ind_data['MACDhist'])
  axs[2].set_title('MACD')

  plt.tight_layout()
  plt.show()
    
def generate_recommendation(ind_data):
  """Generates a basic recommendation based on RSI, MACD, and Bollinger Bands.

  Args:
    data: A pandas DataFrame containing stock data and indicators.

  Returns:
    A string representing the recommendation.
  """

  last_rsi = ind_data['RSI'].iloc[-1]
  last_macd = ind_data['MACD'].iloc[-1]
  last_macd_signal = ind_data['MACDsignal'].iloc[-1]
  ret= calculate_returns(data, stock_dropdown.value, return_type_dropdown.value).loc[date_dropdown.value:,]
  mkt_ret=calculate_returns(data,'BRVM - COMPOSITE', return_type_dropdown.value).loc[date_dropdown.value:,]
  sharpe = round(erk.sharpe_ratio(ret,float(rfr_selection[rfr_dropdown.value])/100,int(per_selection[per_dropdown.value])),5)
  rf_per_period = (1+float(rfr_selection[rfr_dropdown.value])/100)**(1/int(per_selection[per_dropdown.value]))-1
  excess_stock_return = pd.Series(ret - rf_per_period)
  excess_market_return = pd.Series(mkt_ret - rf_per_period)
  beta= stats.linregress(excess_market_return.dropna(), excess_stock_return.dropna())[0]
  jensen_alpha = round(excess_stock_return.mean() - beta * excess_market_return.mean(),5)

# Recommendation
  if last_rsi > 70 and last_macd > last_macd_signal and jensen_alpha > 0 and sharpe > 1:
    return f"Signal haussier fort : Envisagez d'acheter le titre {stock_dropdown.value}."
  elif last_rsi < 30 and last_macd < last_macd_signal and jensen_alpha < 0 and sharpe < 1:
    return f"Signal baissier fort : Envisager de vendre le titre {stock_dropdown.value}."
  elif last_rsi > 70 or last_macd > last_macd_signal:
    return f"Situation de fort achat, soyez prudents sur le titre {stock_dropdown.value}."
  elif last_rsi < 30 or last_macd < last_macd_signal:
    return f"Situation de forte vente, opportunité d'achat potentielle sur le titre {stock_dropdown.value}."
  elif jensen_alpha > 0 and sharpe > 1:
    return f"Performances intéressantes, envisager de la conservation du titre {stock_dropdown.value}."
  elif jensen_alpha < 0 and sharpe < 1:
    return f"Performances insuffisantes, envisager de vendre le titre {stock_dropdown.value}."
  else:
    return f"Nous restons neutre sur le titre {stock_dropdown.value}, une analyse plus approfondie est nécessaire."


def indicators():
    indicators_df = calculate_indicators(data.copy())
    plot_indicators(indicators_df)
    recommendation = generate_recommendation(indicators_df)
    print("RECOMMENDATION:",recommendation)

ind_button = Button(description="Recommendation CT")
ind_button.style.button_color = 'red'
ind_button.layout.width = '210px'
ind_button.layout.height = '40px'

output15 = widgets.Output()

def on_button_clicked15(b):
 with output15:
    output15.clear_output()
    indicators()
    
ind_button.on_click(on_button_clicked15)    
    
# Create an HBox layout to arrange them horizontally
hbox = HBox(children=[ind_button,rend_button,vol_button,downside_button,performance_button,stat_button,dist_button])

display(return_type_dropdown, stock_dropdown,per_dropdown,date_dropdown,rfr_dropdown,hbox, output9, output10, output11, output12, output13, output14,output15)

Dropdown(description='Type de rendement:', options=('Rendements Logarithmiques', 'Rendements Simples'), value=…

Dropdown(description='Actions:', options=('CABC', 'FTSC', 'NEIC', 'NTLC', 'SEMC', 'SIVC', 'SLBC', 'SMBC', 'STB…

Dropdown(description='Périodicité:', options=('Journalier', 'Hebdomadaire', 'Mensuelle'), value='Journalier')

Dropdown(description='Filtre Date:', index=12, options=('2035', '2034', '2033', '2032', '2031', '2030', '2029'…

Dropdown(description='Taux sans risque:', index=10, options=('5.55', '5.65', '5.70', '5.80', '5.85', '5.90', '…

Output()

Output()

Output()

# 2- PLATEFORME D'ANALYSE GROUPÉE DES ACTIONS

Ce document sert à effectuer une analyse comparative des mesures statistiques des titres côtés sur la BRVM. 

In [ ]:
# Dropdown pour la sélection du secteur
data_selection = {
    "Tous les Secteurs": ['CABC','FTSC','NEIC','NTLC','SEMC','SIVC','SLBC','SMBC','STBC','TTRC','UNLC','UNXC','CIEC','ONTBF','ORAC','SDCC','SNTS','BICC','BOAB','BOABF','BOAC','BOAM','BOAN','BOAS','CBIBF','ECOC','ETIT','NSBC','ORGT','SAFC','SGBC','SIBC','SDSC','SVOC','PALC','SCRC','SICC','SOGC','SPHC','ABJC','BNBC','CFAC','PRSC','SHEC','TTLC','TTLS','STAC'],
    "Secteur Finance": ['BOAB','BOABF','BOAC','BOAM','BOAN','BOAS','BICC','CBIBF','ECOC', 'ETIT','NSBC','ORGT','SAFC','SGBC','SIBC','BRVM - FINANCE'],
    "Secteur Services Publics": ['CIEC', 'ONTBF','ORAC','SDCC','SNTS','BRVM - SERVICES PUBLICS'],
    "Secteur Industrie":['SIVC','SEMC','FTSC','NEIC','NTLC','CABC','STBC','SMBC','SLBC','UNLC','UNXC','BRVM - INDUSTRIE'],
    "Secteur Agriculture": ['PALC','SPHC','SICC','SOGC','SCRC','BRVM - AGRICULTURE'],
    "Secteur Distribution": ['BNBC','CFAC','ABJC','TTLC','TTLS','SHEC','PRSC','BRVM - DISTRIBUTION'],
    "Secteur Transport": ['SDSC','SVOC','BRVM - TRANSPORT'],
    "Secteur Autres": ['STAC BC EQUITY','BRVM - AUTRES SECTEURS'],
}

data_options = list(data_selection.keys()) 
data_dropdown = Dropdown(
    description="Secteurs:",
    options=data_options,
    value=data_options[3], 
)


# Definition fonction Log_return et return

def calculate_returns(data, stock, return_type):
  if return_type == 'Rendements Logarithmiques':
    
    returns = np.log(data.loc[:,stock].dropna() / data.loc[:,stock].dropna().shift(1))
  elif return_type == 'Rendements Simples':
    returns = data.loc[:,stock].dropna().pct_change()
  else:
    raise ValueError("Type de rendement non applicable")
  return returns

return_type_options = ['Rendements Logarithmiques', 'Rendements Simples']
return_type_dropdown = widgets.Dropdown(options=return_type_options, description='Type de rendement:')


# Dropdown pour la sélection de la périodicité des données
per_selection = {
    "Journalier":'252',
    "Hebdomadaire":'52', 
    "Mensuelle": '12', 
}


per_options = list(per_selection.keys()) 
per_dropdown = Dropdown(
    description="Périodicité:",
    options=per_options,
    value=per_options[0], 
)

# Dropdown pour la sélection du taux sans risque 
rfr_selection = {
    "5.55":'5.55',
    "5.65":'5.65',
    "5.70":'5.70',
    "5.80":'5.80',
    "5.85":'5.85',
    "5.90":'5.90',
    "5.95":'5.95', 
    "6.00":'6.00', 
    "6.05":'6.05',
    "6.10":'6.10',
    "6.15":'6.15',
    "6.20":'6.20',
    "6.25":'6.25',
    "6.30":'6.30',
    "6.35":'6.35',
    "6.40":'6.40',
    "6.45":'6.45',
    "6.50":'6.50',
    "6.55":'6.55',
    "6.60":'6.60',
    "6.65":'6.65',
    "6.70":'6.70',
    "6.75":'6.75',
    "6.80":'6.80',
    "6.85":'6.85',
    "6.90":'6.90',
}

rfr_options = list(rfr_selection.keys()) 
rfr_dropdown = Dropdown(
    description="Taux sans risque:",
    options=rfr_options,
    value=rfr_options[10], 
)


# Dropdown pour la selection de la date à considérer

# Dropdown pour la sélection de la date
date_selection = {
    "2035":'2035',
    "2034":'2034',
    "2033":'2033',
    "2032":'2032',
    "2031":'2031',
    "2030":'2030',
    "2029":'2029', 
    "2028":'2028', 
    "2027":'2027',
    "2026":'2026',
    "2025":'2025',
    "2024":'2024',
    "2023":'2023',
    "2022":'2022',
    "2021":'2021',
    "2020":'2020',
    "2019":'2019',
    "2018":'2018', 
    "2017":'2017', 
    "2016":'2016',
    "2015":'2015',
    "2014":'2014',
    "2013":'2013',
    "2012":'2012',
    "2011":'2011',
    "2010":'2010',
    "2009":'2009',
    "2008":'2008',
    "2007":'2007',
    "2006":'2006',
    "2005":'2005',
    "2004":'2004',
}

date_options = list(date_selection.keys()) 
date_dropdown = Dropdown(
    description="Filtre Date:",
    options=date_options,
    value=date_options[12], 
)



# Calcul des mesures statistiques

def rend2():
    ret= calculate_returns(data, data_selection[data_dropdown.value], return_type_dropdown.value).loc[date_dropdown.value:,]
    mean_ret= (ret.mean()*100).round(5)
    ann_ret=(erk.annualize_rets(ret,int(per_selection[per_dropdown.value]))*100).round(5)
    comp_ret=(((ret+1).prod()-1)*100).round(5)
    print(f"Rendement moyen {per_dropdown.value} des titres (en %): \n{mean_ret.sort_values(ascending=False)}")
    print("\n")
    print(f"Le titre du {data_dropdown.value} avec le rendement moyen le plus elevée est {mean_ret.idxmax()}, celui dont le rendement moyen est la plus faible est {mean_ret.idxmin()} pour une base de donnée débutant en {date_dropdown.value}.")
    print("\n")
    print(f"Rendement moyen annuel des titres (en %): \n{ann_ret.sort_values(ascending=False)}")
    print("\n")
    print(f"Le titre du {data_dropdown.value} avec le rendement annuel le plus elevée est {ann_ret.idxmax()}, celui dont le rendement annuel est la plus faible est {ann_ret.idxmin()} pour une base de donnée débutant en {date_dropdown.value}.")
    print("\n")
    print(f"Rendement géométrique des titres(en %):  \n{comp_ret.sort_values(ascending=False)}")
    print("\n")
    
 # Dataframe et Excel export 
    rend_df = pd.DataFrame({
    f'Rendement moyen {per_dropdown.value} des titres (en %)': mean_ret,
    'Rendement moyen annuel des titres (en %)': ann_ret,
    'Rendement géométrique des titres(en %)': comp_ret,
})
    rend_df.to_excel('Rendements.xlsx', index=True)
    
    
rend2_button = widgets.Button(description="Rendements",button_style='info')
rend2_button.layout.width = '210px'
rend2_button.layout.height = '40px'

output16 = widgets.Output()

    
def on_button_clicked16(b):
 with output16:
    output16.clear_output()
    rend2()
      
rend2_button.on_click(on_button_clicked16)

def volatility2():
    ret= calculate_returns(data, data_selection[data_dropdown.value], return_type_dropdown.value).loc[date_dropdown.value:,]
    vol = round(ret.std()*100,5)
    ann_vol=round(erk.annualize_vol(ret,int(per_selection[per_dropdown.value]))*100,5)
    print(f"Volatilité Moyenne {per_dropdown.value} des titres (en %): \n{vol.sort_values(ascending=False)}")
    print("\n")
    print(f"Le titre du {data_dropdown.value} avec la volatilité moyenne la plus elevée est {vol.idxmax()}, celui dont la volatilité est la plus faible est {vol.idxmin()} pour une base de donnée débutant en {date_dropdown.value}.")
    print("\n")
    print(f"Volatilité Annuelle des titres (en %): \n{ann_vol.sort_values(ascending=False)}")
    print("\n")
    print(f"Le titre du {data_dropdown.value} avec la volatilité annuelle la plus elevée est {ann_vol.idxmax()}, celui dont la volatilité annuelle est la plus faible est {ann_vol.idxmin()} pour une base de donnée débutant en {date_dropdown.value}.")
    print("\n")

# Dataframe et Excel export 
    vol_df = pd.DataFrame({
    f'Volatilité Moyenne {per_dropdown.value} des titres (en %)':vol,
    'Volatilité Annuelle des titres (en %)': ann_vol,
})
    vol_df.to_excel('Volatilités.xlsx', index=True)
   
    
vol2_button = Button(description="Volatilités", button_style='info')
vol2_button.layout.width = '210px'
vol2_button.layout.height = '40px'
output17 = widgets.Output()

def on_button_clicked17(b):
 with output17:
    output17.clear_output()
    volatility2()
      
vol2_button.on_click(on_button_clicked17)

def downside2():
    ret= calculate_returns(data, data_selection[data_dropdown.value], return_type_dropdown.value).loc[date_dropdown.value:,]
    semidev = round(erk.semideviation(ret)*100,5)
    Var95= round(erk.var_gaussian(ret,level=5, modified=True)*100,5)
    Var99=round(erk.var_gaussian(ret,level=1, modified=True)*100,5)
    is_beyond95 = ret <= -erk.var_gaussian(ret, level=5, modified=True)
    shortfall95=round(-((0.05*ret[is_beyond95].mean())-0.95*erk.var_gaussian(ret, level=5, modified=True))*100,5)
    is_beyond99 = ret <= -erk.var_gaussian(ret, level=1, modified=True)
    shortfall99=round(-((0.01*ret[is_beyond99].mean())-0.99*erk.var_gaussian(ret, level=1, modified=True))*100,5)
    
    # DRAWDOWN 
    col=ret.columns
    my_list1=[]
    for i in col: 
        drawdown=erk.drawdown(ret.loc[:,i])
        mean_drawdown=(drawdown.loc[:,'Drawdown']).min()*100*-1
        my_list1.append(mean_drawdown)
        draw_series=pd.Series(my_list1, name='')
        draw_data=pd.DataFrame(draw_series)
    df1=ret.columns
    table= draw_data.set_index(ret.columns)
    table 
        
        
    print(f"Semideviation {per_dropdown.value} des titres (en %):\n{semidev.sort_values(ascending=False)}")
    print("\n")
    print(f"Le titre du {data_dropdown.value} avec la semidéviation la plus elevée est {semidev.idxmax()}, celui dont la semidéviation est la plus faible est {semidev.idxmin()} pour une base de donnée débutant en {date_dropdown.value}.")
    print("\n")
    print(f"Value at Risk {per_dropdown.value} seuil 5% (en %) des titres: \n{Var95.sort_values(ascending=False)}")
    print("\n")
    print(f"Le titre du {data_dropdown.value} avec la VaR seuil 5% la plus elevée est {Var95.idxmax()}, celui dont la VaR est la plus faible est {Var95.idxmin()} pour une base de donnée débutant en {date_dropdown.value}.")
    print("\n")
    print(f"Expected Shortfall {per_dropdown.value} seuil 5% (en %) des titres:\n{shortfall95.sort_values(ascending=False)}")
    print("\n")
    print(f"Value at Risk {per_dropdown.value} seuil 1% (en %) des titres:\n{Var99.sort_values(ascending=False)}")
    print("\n")
    print(f"Le titre du {data_dropdown.value} avec la VaR seuil 1% la plus elevée est {Var99.idxmax()}, celui dont la VaR est la plus faible est {Var99.idxmin()} pour une base de donnée débutant en {date_dropdown.value}.")
    print("\n")
    print(f"Expected Shortfall {per_dropdown.value} seuil 1% (en %) des titres:\n{shortfall99.sort_values(ascending=False)}")
    print("\n")
    print(f"Maximum Drawdown {per_dropdown.value} (en %) des titres:", table)
    print("\n")
    print(f"Le titre du {data_dropdown.value} avec le MaxDrawdown le plus elevé est {table.idxmax().values[0]}, celui dont le maximum drawdown est la plus faible est {table.idxmin().values[0]} pour une base de donnée débutant en {date_dropdown.value}.")
    print("\n")

    # Dataframe et Excel export 
    ris_baisse_df = pd.DataFrame({
    f'Semideviation {per_dropdown.value} des titres (en %)':semidev,
    f'Value at Risk {per_dropdown.value} seuil 5% (en %) des titres': Var95,
    f'Expected Shortfall {per_dropdown.value} seuil 5% (en %) des titres': shortfall95, 
    f'Value at Risk {per_dropdown.value} seuil 1% (en %) des titres': Var99,
    f'Expected Shortfall {per_dropdown.value} seuil 1% (en %) des titres': shortfall99,
})
    ris_baisse_df.to_excel('Mesures de risque de baisse.xlsx', index=True)
    table.to_excel('MaxDrawdown.xlsx', index=True)


downside2_button = Button(description="Mesure de risque de baisse",button_style='primary')
downside2_button.layout.width = '210px'
downside2_button.layout.height = '40px'

output18 = widgets.Output()

def on_button_clicked18(b):
 with output18:
    output18.clear_output()
    downside2()
      
downside2_button.on_click(on_button_clicked18)


def performance2():
    ret= calculate_returns(data, data_selection[data_dropdown.value], return_type_dropdown.value).loc[date_dropdown.value:,]
    mkt_ret=calculate_returns(data,'BRVM - COMPOSITE', return_type_dropdown.value).loc[date_dropdown.value:,]
    sharpe = round(erk.sharpe_ratio(ret,float(rfr_selection[rfr_dropdown.value])/100,int(per_selection[per_dropdown.value])),5)
    rf_per_period = (1+float(rfr_selection[rfr_dropdown.value])/100)**(1/int(per_selection[per_dropdown.value]))-1
    excess_market_return = pd.Series(mkt_ret - rf_per_period)
    
    # Jensen alpha
    col1=ret.columns
    my_list2=[]
    for i in col1:
        excess_stock_return = pd.Series(ret.loc[:,i] - rf_per_period)
        beta= stats.linregress(excess_market_return.dropna(), excess_stock_return.dropna())[0]
        jensen_alpha = round(excess_stock_return.mean() - beta * excess_market_return.mean(),7)*100
        my_list2.append(jensen_alpha)
        jen_series=pd.Series(my_list2, name=' ')
        jen_data=pd.DataFrame(jen_series)
    table2= jen_data.set_index(ret.columns)
    table2
    
 # Treynor ratio
    col2=ret.columns
    my_list3=[]
    for i in col2:
        excess_stock_return = pd.Series(ret.loc[:,i] - rf_per_period)
        model = sm.OLS(excess_stock_return.dropna(), sm.add_constant(excess_market_return.dropna()))
        beta_est = model.fit().params[1] 
        treynor_ratio =(excess_stock_return.mean()) / beta_est
        my_list3.append(treynor_ratio)
        treynor_series=pd.Series(my_list3, name=' ')
        treynor_data=pd.DataFrame(treynor_series)
        t_data= treynor_data.astype(float)
    table3= t_data.set_index(ret.columns)
    table3
    
    # Sortino ratio
    col3=ret.columns
    my_list4=[]
    for i in col3:
        excess_stock_return = pd.Series(ret.loc[:,i] - rf_per_period)
        downside_risk = np.sqrt(np.mean(excess_stock_return[excess_market_return < 0] ** 2))
        if downside_risk == 0:
            return None 
        sortino_ratio = (excess_stock_return.mean()) / downside_risk
        my_list4.append(sortino_ratio)
        sortino_series=pd.Series(my_list4, name=' ')
        sortino_data=pd.DataFrame(sortino_series)
        s_data= sortino_data.astype(float)
    table4= s_data.set_index(ret.columns)
    table4
    

    print(f"Ratio de Sharpe des titres:\n{sharpe.sort_values(ascending=False)}")
    print("\n")
    print(f"Le titre du {data_dropdown.value} avec le ratio de sharpe le plus elevé est {sharpe.idxmax()}, celui dont le ratio est le plus faible est {sharpe.idxmin()} pour une base de donnée débutant en {date_dropdown.value}.")
    print("\n")
    print(f"Jensen Alpha {per_dropdown.value} des titres:", table2)
    print("\n")
    print(f"Le titre du {data_dropdown.value} avec le jensen alpha le plus elevé est {table2.idxmax().values[0]}, celui dont le ratio est le plus faible est {table2.idxmin().values[0]} pour une base de donnée débutant en {date_dropdown.value}.")
    print("\n")
    print(f"Ratio de treynor des titres :",table3)
    print("\n")
    print(f"Le titre du {data_dropdown.value} avec le ratio de treynor le plus elevé est {table3.idxmax().values[0]}, celui dont le ratio est le plus faible est {table3.idxmin().values[0]} pour une base de donnée débutant en {date_dropdown.value}.")
    print("\n")
    print("Ratio de Sortino des titres:",table4)
    print("\n")
    print(f"Le titre du {data_dropdown.value} avec le ratio de sortino le plus elevé est {table4.idxmax().values[0]}, celui dont le ratio est le plus faible est {table4.idxmin().values[0]} pour une base de donnée débutant en {date_dropdown.value}.")
    print("\n")
    
    sharpe.to_excel('Ratio de sharpe.xlsx', index=True)
    table2.to_excel('Jensen Alpha.xlsx', index=True)
    table3.to_excel('Ratio de Treynor.xlsx', index=True)
    table4.to_excel('Ratio de Sortino.xlsx', index=True)

    
performance2_button = Button(description="Mesure de performance",button_style='primary')
performance2_button.layout.width = '210px'
performance2_button.layout.height = '40px'

output19 = widgets.Output()

def on_button_clicked19(b):
 with output19:
    output19.clear_output()
    performance2()

performance2_button.on_click(on_button_clicked19) 


# Create an HBox layout to arrange them horizontally
hbox = HBox(children=[rend2_button,vol2_button,downside2_button,performance2_button])

display(return_type_dropdown,data_dropdown,per_dropdown,date_dropdown,rfr_dropdown,hbox,output16,output17,output18,output19)
